In [149]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split as split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from IPython.display import  display


%matplotlib inline

def norm_cols(df):
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    return df


def norm_values(df):
    str_cols = list(df.dtypes[df.dtypes == 'object'].index)
    for col in str_cols:
        df[col] = df[col].str.lower().str.replace(' ', '_')
    return df


def make_binary(df, col, val='yes'):
    df[col] = (df[col] == val).astype(int)
    return df

In [150]:
df = pd.read_csv('../../data/churn/churndata.csv')

In [151]:
print(len(df))
df.head()
df.head().T
df.dtypes

7043


customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [152]:
total_charges = pd.to_numeric(df.TotalCharges, errors='coerce')
df[total_charges.isnull()][['customerID', 'TotalCharges']]

,customerID,TotalCharges
488,4472-LVYGI,
753,3115-CZMZD,
936,5709-LVOEQ,
1082,4367-NUYAO,
1340,1371-DWPAZ,
3331,7644-OMVMY,
3826,3213-VVOLG,
4380,2520-SGTTA,
5218,2923-ARZLG,
6670,4075-WKNIU,


In [153]:
# Normalize everything
df.TotalCharges = pd.to_numeric(df.TotalCharges, errors='coerce')
df.TotalCharges = df.TotalCharges.fillna(0)
df = norm_cols(df)
df = norm_values(df)
df = make_binary(df, 'churn')

In [154]:
# Create data sets
df_train_full, df_test = split(df, test_size=0.20, random_state=1)
df_train, df_val = split(df_train_full, test_size=0.22, random_state=11)

y_train = df_train.churn.values
y_val = df_val.churn.values

del df_train['churn']
del df_val['churn']

In [155]:
# Analyze the data
# Find missing values
df_train_full.isnull().sum()

# Check prediction feature distro
df_train_full.churn.value_counts()

# Check churn rate
global_mean = df_train_full.churn.mean()
round(global_mean, 3)

# Split column types
numerical = ['tenure','monthlycharges','totalcharges']
ignored = ['customerid'] +  numerical
categorical = [col for col in df_train.columns if col not in ignored]

# See number of categorical values
df_train_full[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

In [156]:
# Feature importance analysis
# Get means of churn by gender
female_mean = df_train_full[df_train_full.gender == 'female'].churn.mean()
male_mean = df_train_full[df_train_full.gender == 'male'].churn.mean()
print(female_mean, male_mean) # gender is not important

# Get means of churn by partner yes/no
partner_yes = df_train_full[df_train_full.partner == 'yes'].churn.mean()
partner_no = df_train_full[df_train_full.partner == 'no'].churn.mean()
print(partner_yes, partner_no) # partner is important

# Risk ration
# Just gender
df_group = df_train_full.groupby(by='gender').churn.agg(['mean'])
df_group['diff'] = df_group['mean'] - global_mean
df_group['risk'] = df_group['mean'] / global_mean
df_group

# All categorical cols
for col in categorical:
    df_group = df_train_full.groupby(by=col).churn.agg(['mean'])
    df_group['diff'] = df_group['mean'] - global_mean
    df_group['risk'] = df_group['mean'] / global_mean
    display(df_group)

0.27682403433476394 0.2632135306553911
0.20503330866025166 0.3298090040927694


,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


,mean,diff,risk
seniorcitizen,,,
0,0.242270,-0.027698,0.897403
1,0.413377,0.143409,1.531208


,mean,diff,risk
partner,,,
no,0.329809,0.059841,1.221659
yes,0.205033,-0.064935,0.759472


,mean,diff,risk
dependents,,,
no,0.313760,0.043792,1.162212
yes,0.165666,-0.104302,0.613651


...

,mean,diff,risk
streamingtv,,,
no,0.342832,0.072864,1.269897
no_internet_service,0.077805,-0.192163,0.288201
yes,0.302723,0.032755,1.121328


,mean,diff,risk
streamingmovies,,,
no,0.338906,0.068938,1.255358
no_internet_service,0.077805,-0.192163,0.288201
yes,0.307273,0.037305,1.138182


,mean,diff,risk
contract,,,
month-to-month,0.431701,0.161733,1.599082
one_year,0.120573,-0.149395,0.446621
two_year,0.028274,-0.241694,0.104730


,mean,diff,risk
paperlessbilling,,,
no,0.172071,-0.097897,0.637375
yes,0.338151,0.068183,1.252560


,mean,diff,risk
paymentmethod,,,
bank_transfer_(automatic),0.168171,-0.101797,0.622928
credit_card_(automatic),0.164339,-0.105630,0.608733
electronic_check,0.455890,0.185922,1.688682
mailed_check,0.193870,-0.076098,0.718121


In [157]:
# Mutual information
def calc_mi(series):
    return mutual_info_score(series, df_train_full.churn)

df_mi = df_train_full[categorical].apply(calc_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')
df_mi

# Correlation of numerical cols
df_train_full[numerical].corrwith(df_train_full.churn)

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

In [162]:
# Feature engineering
# One-hot encode categorical cols
# Create a training dict from data where k: col, v: value
train_dict = df_train[categorical + numerical].to_dict(orient='records')

# Check the first...
train_dict[0]

# Create vectorizer and fit it
dictzer = DictVectorizer(sparse=False)
dictzer.fit(train_dict)

# Use it to transform train_dict into the training set
X_train = dictzer.transform(train_dict)

# Check the first...
X_train[0]

# See the len and values of the training cols
names = dictzer.get_feature_names()
print('Total training cols:', len(names))
names

Total training cols: 45


['contract=month-to-month',
 'contract=one_year',
 'contract=two_year',
 'dependents=no',
 'dependents=yes',
 'deviceprotection=no',
 'deviceprotection=no_internet_service',
 'deviceprotection=yes',
 'gender=female',
 'gender=male',
 'internetservice=dsl',
 'internetservice=fiber_optic',
 'internetservice=no',
 'monthlycharges',
 'multiplelines=no',
 'multiplelines=no_phone_service',
 'multiplelines=yes',
 'onlinebackup=no',
 'onlinebackup=no_internet_service',
 'onlinebackup=yes',
 'onlinesecurity=no',
 'onlinesecurity=no_internet_service',
 'onlinesecurity=yes',
 'paperlessbilling=no',
 'paperlessbilling=yes',
 'partner=no',
 'partner=yes',
 'paymentmethod=bank_transfer_(automatic)',
 'paymentmethod=credit_card_(automatic)',
 'paymentmethod=electronic_check',
 'paymentmethod=mailed_check',
 'phoneservice=no',
 'phoneservice=yes',
 'seniorcitizen',
 'streamingmovies=no',
 'streamingmovies=no_internet_service',
 'streamingmovies=yes',
 'streamingtv=no',
 'streamingtv=no_internet_servic